# Implementação do Branch-and-bound

###Objetivo:
Resolver problemas de Programação Inteira, maximizando ou minimizando a função objetivo sujeita a restrições específicas. Para isso, o algoritmo de Branch-and-bound é utilizado para explorar de forma eficiente o espaço de solução, evitando uma busca exaustiva. Ao utilizar técnicas de otimização, como podas e estratégias de seleção de nós, o algoritmo torna-se mais eficiente na busca pela solução ótima.

###Etapas de resolução:

* Inicialização: Cria nó raiz contendo solução vazia e o limite
superior igual a menos infinito;

* Seleção de Nó: Escolher do nó para expandir da lista de nós abertos, utilizando o processo de busca em largura;

* Expansão do Nó: Se a solução do nó selecionado não for uma solução inteira, o nó será ramificado em dois nós filhos, adicionando restrições que fixem uma variável para 0 em um nó filho e para 1 no outro;

* Cálculo do Limite Superior: Calcular o limite superior para cada nó filho usando um solver para resolver o subproblema relaxado(com variáveis que podem ser contínuas, não apenas binárias);

* Atualização da Melhor Solução: Se o limite superior de um nó filho for melhor que a melhor solução atual, atualiza a melhor solução.

* Podas: Podar subárvores que não têm potencial para produzir uma solução melhor do que a melhor solução encontrada até o momento. Isso pode incluir poda por **inviabilidade**(se um nó filho é inviável) e poda por **otimalidade**(se o limite superior do nó filho é pior do que o melhor valor atual);

* Fim de Exploração: O algoritmo termina quando todos os nós tiverem sido explorados ou quando não houver mais nós candidatos a serem explorados que possam melhorar a melhor solução atual.

In [3]:
!pip install mip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 27.5 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.16.0
    Uninstalling cffi-1.16.0:
      Successfully uninstalled cffi-1.16.0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
from mip import *
import numpy as np

In [20]:
def load_data(filename):
    with open(filename, 'r') as file:
        # Primeira linha contém num variáveis e num restrições
        num_vars, num_constraints = map(int, file.readline().split())

        # coeficientes da func objetivo
        objective_coeffs = list(map(int, file.readline().split()))

        # coeficientes das restrições e valores limite
        constraints_coeffs = [list(map(int, line.split())) for line in file]

    return num_vars, num_constraints, objective_coeffs, constraints_coeffs


In [21]:
def create_model(num_vars, num_constraints, objective_coefficients, constraint_coefficients):
    model = Model(sense=MAXIMIZE) # inicialização do modelo

    variables = [model.add_var(var_type="CONTINUOUS", lb=0, ub=1, name="x_" + str(i)) for i in range(num_vars)]
    # adicionando variáveis do tipo contínuas(para o propósito do algorítmo passar pelas soluções até encontrar a sol com vars inteiras)

    model.objective = xsum(objective_coefficients[i]*variables[i] for i in range(num_vars)) # func objetivo

    # adiciona restrições do arquivo ao modelo
    for i in range(num_constraints):
        model += xsum(constraint_coefficients[i][j]*variables[j] for j in range(num_vars)) <= constraint_coefficients[i][-1]

    return model

def solve_model(model): # resolvedor
    model.optimize()
    solution = {}
    solution["objective_value"] = model.objective_value
    solution["variables"] = model.vars

    return solution

# pega o item mais próximo de "value" em um array
def find_nearest(array, value):
    array = np.asarray(array) # apenas para usar a função argmin
    idx = (np.abs(array - value)).argmin() # calcula a dist de cada item do array para "value" e pega o idx do item de menor dist
    return idx

# cria um ponto de ramificação do problema em dois subproblemas(branching)
def split_model(model, variable_values):
    var_to_branch = variable_values[find_nearest([var.x for var in variable_values], 0.5)]
    # escolha da variável a ser fixada na ramificação(a ideia é escolher a variável mais próxima de 0.5, porque ela estaria
    #"indecisa" entre 0 e 1)

    model_0 = model.copy()
    model_0 += var_to_branch == 0 # a var_to_branch é fixada em 0 usando uma restrição adicional

    model_1 = model.copy()
    model_1 += var_to_branch == 1 # a var_to_branch é fixada em 1 usando uma restrição adicional

    return (model_0, model_1)

In [22]:

# func que resolve o modelo a cada passo(branch) e decide se a sol é inviável(retorna infeasibility),
# se é inteira(retorna integrality), se é a melhor solução encontrada até o momento(retorna limit),
# checando o lim primal global com o calculado na execução.
def check_bounds(model):
    global primal_bound
    solution = solve_model(model)

    if solution["objective_value"] is None:
        return 'infeasibility'

    count_integer_vars = sum(1 for var in solution["variables"] if var.x.is_integer())
    if count_integer_vars == len(solution["variables"]):
        return 'integrality'

    if solution["objective_value"] <= primal_bound:
        return 'limit' # sol limitante

    return 'fractional' # pode-se fazer branching



def branch_and_bound(model):
    global optimal_solution
    global primal_bound
    node_queue = [model]

    while node_queue != []:
        model_solution = solve_model(node_queue[0]) # resolve o modelo atual, caso sol inviaável, retorna infeasibility
        bound_check = check_bounds(node_queue[0])
        if bound_check == 'infeasibility' or bound_check == 'limit':
            node_queue.pop(0)
        elif bound_check == 'integrality':
            if model_solution["objective_value"] > primal_bound: # caso bound maior que o atual, atualiza limite
                optimal_solution = node_queue[0] # atualiza a melhor sol encontrada
                primal_bound = model_solution["objective_value"] # atualiza o limite primal global
            node_queue.pop(0)
        elif bound_check == 'fractional': # caso solução fracionária, faz o branching
            branch_models = split_model(node_queue[0], model_solution["variables"])
            node_queue.append(branch_models[0])
            node_queue.append(branch_models[1])
            node_queue.pop(0)


In [24]:

for i in range(1, 5):

  primal_bound = 0
  optimal_solution = None
  # num_vars = 0
  # num_constraints = 0
  # objective_coefficients = []
  # constraint_coefficients = []

  num_vars, num_constraints, objective_coefficients, constraint_coefficients = load_data(f"/content/drive/MyDrive/Universidade/projeto_po_testes/teste{i}.txt")

  model_instance = create_model(num_vars, num_constraints, objective_coefficients, constraint_coefficients)

  branch_and_bound(model_instance)
  solved_model = solve_model(optimal_solution)


  print(f'Teste')
  print("Optimal Solution:")

  for var in solved_model["variables"]:
      print(var.name, ' = ', var.x)

  print("Objective Value:")
  print('Z = ', solved_model["objective_value"])
  print('-------------------------------------------------------------')



Teste
Optimal Solution:
x_0  =  0.0
x_1  =  0.0
x_2  =  0.0
x_3  =  0.0
x_4  =  1.0
x_5  =  1.0
x_6  =  0.0
Objective Value:
Z =  20.0
-------------------------------------------------------------
Teste
Optimal Solution:
x_0  =  0.0
x_1  =  0.0
x_2  =  0.0
x_3  =  0.0
x_4  =  0.0
x_5  =  1.0
x_6  =  1.0
x_7  =  0.0
x_8  =  1.0
Objective Value:
Z =  24.0
-------------------------------------------------------------
Teste
Optimal Solution:
x_0  =  0.0
x_1  =  0.0
x_2  =  1.0
x_3  =  0.0
x_4  =  0.0
x_5  =  0.0
x_6  =  1.0
x_7  =  0.0
x_8  =  0.0
Objective Value:
Z =  19.0
-------------------------------------------------------------
Teste
Optimal Solution:
x_0  =  0.0
x_1  =  0.0
x_2  =  1.0
x_3  =  0.0
x_4  =  0.0
x_5  =  0.0
x_6  =  0.0
x_7  =  0.0
x_8  =  0.0
Objective Value:
Z =  10.0
-------------------------------------------------------------
